In [1]:
# wangling
import numpy as np
import pandas as pd
import feather

# visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# geospatial
import geopandas as gpd
from shapely.geometry import Point

# others
import re
from datetime import datetime
from tqdm import tqdm_notebook as tqdm

/usr/local/var/pyenv/versions/3.7.2/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/var/pyenv/versions/3.7.2/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/var/pyenv/versions/3.7.2/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/var/pyenv/versions/3.7.2/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/var/pyenv/versions/3.7.2/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed,

# load data